In [1]:
# import libraries
import pandas as pd
import os
import glob
import math
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
  
  
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
    df['Transaction'] = df['Transaction'].str.lower()
    
    df_new = pd.DataFrame()
    
    #Dans ce qu'il y a dans la colonne Transaction, on garde uniquement les catégories suivantes : ['exports', 'imports', 'production', 'final consumption', 'losses']
    for trans in ['exports', 'imports', 'production', 'final consumption', 'losses']:
        df_new = df_new.append(df.loc[df['Transaction']==trans], ignore_index = True)
    
    df_sankey = pd.DataFrame(columns=['source','target','value','year','estimation'])
    
    # Convert data in : ['source','target','value','year','estimation']
    # source : Commodity - transaction, target : transaction, value : Conversion_in_kWh, year : Year, estimation : Quantity Footnotes and if NaN = 0
    
    # source : Transaction (Import, Production), target : Total amount of energy, value : sum of Conversion_in_kWh, year : Year, estimation : Quantity Footnotes and if NaN = 0
    
    # source : Total amount of energy, target : Transaction (Consumption, Losses, Exports), value : sum of Conversion_in_kWh, year : Year, estimation : Quantity Footnotes and if NaN = 0
    
    imports = pd.DataFrame(columns=['imports','year','estimation'])
    exports = pd.DataFrame(columns=['exports','year','estimation'])
    production = pd.DataFrame(columns=['production','year','estimation'])
    consumption = pd.DataFrame(columns=['final consumption','year','estimation'])
    losses = pd.DataFrame(columns=['losses','year','estimation'])
    
    for i in df_new.index:
        estimation = 0
        if math.isnan(df_new["Quantity Footnotes"][i]) :
            estimation = 0
        else :
            estimation = df_new["Quantity Footnotes"][i]
        
        if (df_new["Transaction"][i] == 'imports') :
            new_row = pd.DataFrame({'source': df_new["Commodity"][i]+' - '+ df_new["Transaction"][i], 'target':df_new["Transaction"][i], 'value':[df_new["Conversion_in_kWh"][i]], 'year':[df_new["Year"][i]], 'estimation':[estimation]})
        elif (df_new["Transaction"][i] == 'production') :
            new_row = pd.DataFrame({'source': df_new["Commodity"][i]+' - '+ df_new["Transaction"][i], 'target':df_new["Transaction"][i], 'value':[df_new["Conversion_in_kWh"][i]], 'year':[df_new["Year"][i]], 'estimation':[estimation]})
        else : 
            new_row = pd.DataFrame({'source': df_new["Transaction"][i], 'target':df_new["Commodity"][i]+' - '+ df_new["Transaction"][i], 'value':[df_new["Conversion_in_kWh"][i]], 'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
        
        if df_new["Transaction"][i] == 'imports' :
            if df_new["Year"][i] in imports['year'] :
                imports['imports'].loc[imports['year']==df_new["Year"][i]] += df_new["Conversion_in_kWh"][i]
                imports['estimation'].loc[imports['year']==df_new["Year"][i]] += estimation
            else : 
                new_row = pd.DataFrame({'imports': [df_new["Conversion_in_kWh"][i]], 
                        'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
                imports = imports.append(new_row, ignore_index = True)
            
        elif df_new["Transaction"][i] == 'exports' :
            if df_new["Year"][i] in exports['year'] :
                exports['exports'].loc[exports['year']==df_new["Year"][i]] += df_new["Conversion_in_kWh"][i]
                exports['estimation'].loc[exports['year']==df_new["Year"][i]] += estimation
            else : 
                new_row = pd.DataFrame({'exports': [df_new["Conversion_in_kWh"][i]], 
                        'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
                exports = exports.append(new_row, ignore_index = True)
            
        elif df_new["Transaction"][i] == 'production' :
            if df_new["Year"][i] in production['year'] :
                production['production'].loc[production['year']==df_new["Year"][i]] += df_new["Conversion_in_kWh"][i]
                production['estimation'].loc[production['year']==df_new["Year"][i]] += estimation
            else : 
                new_row = pd.DataFrame({'production': [df_new["Conversion_in_kWh"][i]], 
                        'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
                production = production.append(new_row, ignore_index = True)
            
        elif df_new["Transaction"][i] == 'final consumption' :
            if df_new["Year"][i] in consumption['year'] :
                consumption['final consumption'].loc[consumption['year']==df_new["Year"][i]] += df_new["Conversion_in_kWh"][i]
                consumption['estimation'].loc[consumption['year']==df_new["Year"][i]] += estimation
            else : 
                new_row = pd.DataFrame({'final consumption': [df_new["Conversion_in_kWh"][i]], 
                        'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
                consumption = consumption.append(new_row, ignore_index = True)
            
        elif df_new["Transaction"][i] == 'losses' :
            if df_new["Year"][i] in losses['year'] :
                losses['losses'].loc[losses['year']==df_new["Year"][i]] += df_new["Conversion_in_kWh"][i]
                losses['estimation'].loc[losses['year']==df_new["Year"][i]] += estimation
            else : 
                new_row = pd.DataFrame({'losses': [df_new["Conversion_in_kWh"][i]], 
                        'year':[df_new["Year"][i]], 'estimation':[estimation]})
        
                losses = losses.append(new_row, ignore_index = True)
    
    for i in imports.index:
        new_row = pd.DataFrame({'source': 'imports', 'target': 'Total amount of energy', 'value': [imports['imports'][i]], 
                        'year':[imports['year'][i]], 'estimation':[imports['estimation'][i]]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
        
    for i in production.index:
        new_row = pd.DataFrame({'source': 'production', 'target': 'Total amount of energy', 'value': [production['production'][i]], 
                        'year':[production['year'][i]], 'estimation':[production['estimation'][i]]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
        
    for i in exports.index:
        new_row = pd.DataFrame({'source': 'Total amount of energy', 'target': 'exports', 'value': [exports['exports'][i]], 
                        'year':[exports['year'][i]], 'estimation':[exports['estimation'][i]]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
        
    for i in consumption.index:
        new_row = pd.DataFrame({'source': 'Total amount of energy', 'target': 'final consumption', 'value': [consumption['final consumption'][i]], 
                        'year':[consumption['year'][i]], 'estimation':[consumption['estimation'][i]]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
        
    for i in losses.index:
        new_row = pd.DataFrame({'source': 'Total amount of energy', 'target': 'losses', 'value': [losses['losses'][i]], 
                        'year':[losses['year'][i]], 'estimation':[losses['estimation'][i]]})
        
        df_sankey = df_sankey.append(new_row, ignore_index = True)
    
    
    df_sankey.to_csv('0Sankey_' + df['Country or Area'].iloc[0] + '.csv')

    